In [1]:
import numpy as np

### Forward Pass

1. **Initialization**: 
   - Let $T$ be the number of time steps in the input sequence.
   - Initialize the hidden state matrix $H$ with zeros. It has dimensions `[hidden_size, T]`.
   - Initialize the output predictions matrix $Y_{pred}$ with zeros. It has dimensions `[output_size, T]`.

2. **For each time step $t$ from 1 to $T$**:
   - $x_t$: The input at time step $t$, with shape `[input_size, 1]`.
   - $h_{prev}$: The previous hidden state. For $t = 1$, this is the initial hidden state passed to the function; for $t > 1$, it's the hidden state from the previous time step.
   - Compute the current hidden state $h_t$ using the formula:
     $$
     h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{prev} + b_h)
     $$
     where $W_{xh}$ are the weights connecting the input to the hidden layer, $W_{hh}$ are the weights connecting the previous hidden state to the current hidden state, and $b_h$ is the bias for the hidden state.
   - Compute the output $y_{pred, t}$ at time $t$ using the formula:
     $$
     y_{pred, t} = W_{hy} \cdot h_t + b_y
     $$
     where $W_{hy}$ are the weights connecting the hidden state to the output, and $b_y$ is the output bias.

3. **Return**: The function returns $Y_{pred}$, the matrix of output predictions for each time step, and the final hidden state $h_T$.

### Backward Pass

1. **Initialization**:
   - Initialize gradients for weights and biases ($dW_{xh}$, $dW_{hh}$, $dW_{hy}$, $db_h$, $db_y$) with zeros, matching the dimensions of the corresponding parameters.
   - Initialize $dh_{next}$ with zeros, which represents the gradient of the loss with respect to the next hidden state (initially, the hidden state of the last time step).

2. **For each time step $t$ from $T$ down to 1**:
   - Compute the gradient of the loss with respect to the output predictions at time $t$, $dy_{t} = y_{pred, t} - targets_t$.
   - Update $dW_{hy}$ and $db_y$ with the gradients contributed by time step $t$:
     $$
     dW_{hy} += dy_{t} \cdot h_{t}^T
     $$
     $$
     db_y += dy_{t}
     $$
   - Propagate the gradients back to the hidden state:
     $$
     dh = W_{hy}^T \cdot dy_{t} + dh_{next}
     $$
   - Compute the gradient of the loss with respect to the raw (pre-tanh) hidden state ($dh_{raw}$) by applying the derivative of the $\tanh$ function:
     $$
     dh_{raw} = (1 - h_{t}^2) \cdot dh
     $$
   - Update $dW_{xh}$, $dW_{hh}$, and $db_h$ with the gradients contributed by time step $t$.
   - Propagate $dh_{raw}$ backward through time by setting $dh_{next} = W_{hh}^T \cdot dh_{raw}$.

3. **Gradient Clipping**: To mitigate the exploding gradients problem, clip the gradients ($dW_{xh}$, $dW_{hh}$, $dW_{hy}$, $db_h$, $db_y$) to a specified range (e.g., $[-5, 5]$).

4. **Return**: The gradients ($dW_{xh}$, $dW_{hh}$, $dW_{hy}$, $db_h$, $db_y$) are returned for use in the parameter update step.

In [2]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs, hidden):
        T = inputs.shape[1]
        h = np.zeros((self.hidden_size, T))
        y_pred = np.zeros((self.output_size, T))
        
        for t in range(T):
            xt = inputs[:, t:t+1]
            h_prev = h[:, t-1:t] if t > 0 else hidden
            h[:, t:t+1] = np.tanh(np.dot(self.Wxh, xt) + np.dot(self.Whh, h_prev) + self.bh)
            y_pred[:, t:t+1] = np.dot(self.Why, h[:, t:t+1]) + self.by
        
        return y_pred, h
    
    def backward(self, inputs, targets, y_pred, h):
        T = inputs.shape[1]
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        dhnext = np.zeros_like(h[:, 0:1])
        
        dy = y_pred - targets
        
        for t in reversed(range(T)):
            dWhy += np.dot(dy[:, t:t+1], h[:, t:t+1].T)
            dby += dy[:, t:t+1]
            dh = np.dot(self.Why.T, dy[:, t:t+1]) + dhnext  # backprop into h
            dhraw = (1 - h[:, t:t+1] ** 2) * dh  # through tanh nonlinearity
            if t > 0:
                dWhh += np.dot(dhraw, h[:, t-1:t].T)
            dWxh += np.dot(dhraw, inputs[:, t:t+1].T)
            dbh += dhraw
            dhnext = np.dot(self.Whh.T, dhraw)
        
        # clip
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)
        
        return dWxh, dWhh, dWhy, dbh, dby
    
    def loss(self, y_pred, targets):
        return 0.5 * np.sum((y_pred - targets) ** 2)
    
    def train(self, inputs, targets, learning_rate):
        hidden = np.zeros((self.hidden_size, 1))
        y_pred, h = self.forward(inputs, hidden)
        dWxh, dWhh, dWhy, dbh, dby = self.backward(inputs, targets, y_pred, h)
        
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby
        
        return self.loss(y_pred, targets)

In [ ]:
# multilayer
class RNNLayer:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        
    def forward(self, xt, h_prev):
        return np.tanh(np.dot(self.Wxh, xt) + np.dot(self.Whh, h_prev) + self.bh)
    
    def backward(self, dhnext, dh, h_prev, xt):
        dhraw = (1 - h_prev ** 2) * dh
        dWxh = np.dot(dhraw, xt.T)
        dWhh = np.dot(dhraw, h_prev.T)
        dbh = dhraw
        dhnext = np.dot(self.Whh.T, dhraw) + dhnext
        return dWxh, dWhh, dbh, dhnext

class RNNBase:
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.layers = [RNNLayer(input_size if i == 0 else hidden_size, hidden_size) for i in range(num_layers)]
        self.Why = np.random.randn(output_size, hidden_size) * 0.01
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        T = inputs.shape[1]
        h = np.zeros((self.num_layers, self.hidden_size, T))
        y_pred = np.zeros((self.output_size, T))
        
        for t in range(T):
            xt = inputs[:, t:t+1]
            for l, layer in enumerate(self.layers):
                h_prev = h[l, :, t-1:t] if t > 0 else np.zeros((self.hidden_size, 1))
                h[l, :, t:t+1] = layer.forward(xt, h_prev)
                xt = h[l, :, t:t+1]
                
            y_pred[:, t:t+1] = np.dot(self.Why, h[-1, :, t:t+1]) + self.by
        
        return y_pred, h
    
    def backward(self, inputs, targets, y_pred, h):
        T = inputs.shape[1]
        dWxh, dWhh, dWhy = [np.zeros_like(layer.Wxh) for layer in self.layers], [np.zeros_like(layer.Whh) for layer in self.layers], np.zeros_like(self.Why)
        dbh, dby = [np.zeros_like(layer.bh) for layer in self.layers], np.zeros_like(self.by)
        dhnext = [np.zeros((self.hidden_size, 1)) for _ in range(self.num_layers)]
        
        dy = y_pred - targets
        
        for t in reversed(range(T)):
            dWhy += np.dot(dy[:, t:t+1], h[-1, :, t:t+1].T)
            dby += dy[:, t:t+1]
            dh = np.dot(self.Why.T, dy[:, t:t+1])
            
            for l in reversed(range(self.num_layers)):
                layer = self.layers[l]
                h_prev = h[l, :, t-1:t] if t > 0 else np.zeros((self.hidden_size, 1))
                xt = inputs[:, t:t+1] if l == 0 else h[l-1, :, t:t+1]
                dWxh_l, dWhh_l, dbh_l, dhnext[l] = layer.backward(dhnext[l], dh, h_prev, xt)
                dWxh[l] += dWxh_l
                dWhh[l] += dWhh_l
                dbh[l] += dbh_l
                dh = dhnext[l]
                
        for dparam in dWxh + dWhh + [dWhy] + dbh + [dby]:
            np.clip(dparam, -5, 5, out=dparam)
        
        return dWxh, dWhh, dWhy, dbh, dby
    
    def loss(self, y_pred, targets):
        return 0.5 * np.sum((y_pred - targets) ** 2)
    
    def train(self, inputs, targets, learning_rate):
        y_pred, h = self.forward(inputs)
        dWxh, dWhh, dWhy, dbh, dby = self.backward(inputs, targets, y_pred, h)
        
        for l, layer in enumerate(self.layers):
            layer.Wxh -= learning_rate * dWxh[l]
            layer.Whh -= learning_rate * dWhh[l]
            layer.bh -= learning_rate * dbh[l]
        self.Why -= learning_rate * dWhy
        self.by -= learning_rate * dby
        
        return self.loss(y_pred, targets)

In [3]:
def generate_timeseries(length, num_series, noise_level=0.00001, seasonal_period=1000):
    t = np.arange(length)
    series = []
    
    for i in range(num_series):
        frequency = np.random.uniform(0.01, 0.1)
        sinusoid = np.sin(2 * np.pi * frequency * t)
        brownian = np.cumsum(np.random.normal(0, 0.1, length))
        seasonal = np.sin(2 * np.pi * t / seasonal_period)
        series_i = sinusoid + brownian + seasonal
        noise = np.random.normal(0, noise_level, length)
        series_i += noise
        series.append(series_i)
    
    return np.array(series).T  # to match expected shape

In [4]:
series = generate_timeseries(length=1000, num_series=1, noise_level=0.01, seasonal_period=100)
series

array([[-4.83978066e-02],
       [ 3.64129382e-01],
       [ 8.07742481e-01],
       [ 1.04409221e+00],
       [ 1.11282225e+00],
       [ 1.31845818e+00],
       [ 1.35312699e+00],
       [ 9.55151591e-01],
       [ 6.93448987e-01],
       [ 4.36926413e-01],
       [ 2.18981002e-01],
       [-2.44048666e-02],
       [ 1.39819109e-02],
       [ 3.28092494e-01],
       [ 5.28171116e-01],
       [ 8.75689348e-01],
       [ 1.18912375e+00],
       [ 1.63144205e+00],
       [ 1.91496246e+00],
       [ 1.87552237e+00],
       [ 1.74829209e+00],
       [ 1.59699279e+00],
       [ 1.29571585e+00],
       [ 9.23800174e-01],
       [ 3.85324800e-01],
       [-3.52041748e-02],
       [-1.88295843e-01],
       [-2.12391252e-01],
       [ 4.35548010e-03],
       [ 3.55400660e-01],
       [ 6.54140691e-01],
       [ 1.07505875e+00],
       [ 1.35333781e+00],
       [ 1.60469511e+00],
       [ 1.80012333e+00],
       [ 1.56837112e+00],
       [ 1.32084427e+00],
       [ 8.79279327e-01],
       [ 3.9

In [5]:
series.shape[0]

1000

In [10]:
input_size = 1
output_size = 1
sequence_length = 10
num_sequences = series.shape[0] - sequence_length
num_sequences

990

In [7]:
series.T

array([[-4.83978066e-02,  3.64129382e-01,  8.07742481e-01,
         1.04409221e+00,  1.11282225e+00,  1.31845818e+00,
         1.35312699e+00,  9.55151591e-01,  6.93448987e-01,
         4.36926413e-01,  2.18981002e-01, -2.44048666e-02,
         1.39819109e-02,  3.28092494e-01,  5.28171116e-01,
         8.75689348e-01,  1.18912375e+00,  1.63144205e+00,
         1.91496246e+00,  1.87552237e+00,  1.74829209e+00,
         1.59699279e+00,  1.29571585e+00,  9.23800174e-01,
         3.85324800e-01, -3.52041748e-02, -1.88295843e-01,
        -2.12391252e-01,  4.35548010e-03,  3.55400660e-01,
         6.54140691e-01,  1.07505875e+00,  1.35333781e+00,
         1.60469511e+00,  1.80012333e+00,  1.56837112e+00,
         1.32084427e+00,  8.79279327e-01,  3.95415643e-01,
        -1.39710374e-01, -6.68133414e-01, -9.12377962e-01,
        -1.00025406e+00, -1.02074362e+00, -8.22540383e-01,
        -5.22233481e-01, -3.26523953e-01,  2.36586360e-01,
         5.10288679e-01,  5.82630294e-01,  4.06640192e-0

In [12]:
inputs = []
targets = []
for i in range(num_sequences):
    input_seq = series[i:i+sequence_length].reshape(1, -1)
    target_seq = series[i+1:i+1+sequence_length].reshape(1, -1)  # the next timestep
    inputs.append(input_seq)
    targets.append(target_seq)
    
inputs = np.array(inputs)
targets = np.array(targets)

hidden_size = 50
rnn = RNN(input_size, hidden_size, output_size, 2)

num_epochs = 20
learning_rate = 0.01

for epoch in range(num_epochs):
    epoch_loss = 0
    for i in range(len(inputs)):
        input_seq = inputs[i]
        target_seq = targets[i]
        loss = rnn.train(input_seq, target_seq, learning_rate)
        epoch_loss += loss
    average_loss = epoch_loss / len(inputs)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")

Epoch 1/20, Average Loss: 1.9186
Epoch 2/20, Average Loss: 1.0094
Epoch 3/20, Average Loss: 0.9342
Epoch 4/20, Average Loss: 1.2599
Epoch 5/20, Average Loss: 1.1023
Epoch 6/20, Average Loss: 1.2267
Epoch 7/20, Average Loss: 1.3849
Epoch 8/20, Average Loss: 1.1499
Epoch 9/20, Average Loss: 1.0503
Epoch 10/20, Average Loss: 0.9019
Epoch 11/20, Average Loss: 0.9434
Epoch 12/20, Average Loss: 0.8579
Epoch 13/20, Average Loss: 1.4417
Epoch 14/20, Average Loss: 0.9817
Epoch 15/20, Average Loss: 0.8739
Epoch 16/20, Average Loss: 0.9215
Epoch 17/20, Average Loss: 1.0640
Epoch 18/20, Average Loss: 0.9064
Epoch 19/20, Average Loss: 1.0281
Epoch 20/20, Average Loss: 1.0614
